## Basic Model Setting

In [ ]:
# install the package for videogenerator used in training
!pip install keras-video-generators

In [ ]:
#get access from google drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /

In [ ]:
#set the directory for training data
train_dir='/content/gdrive/My Drive/CSCE636_Deep_Learning/Model/Custom_Dataset/Custom_Dataset4/{classname}/*'

In [ ]:
import keras
from keras_video import VideoFrameGenerator

In [ ]:
#global parameters for training
size = (224, 224)    # the input image size is (224,224,4)
channels = 3
N_frames = 8       # each video is cut into 8 frames
Batch_size = 8
classes=2          #two classes: one for target action, the other is for other actions
train_validate_split=0.3  #30% data used for validation  

In [ ]:
# for data augmentation
from keras_preprocessing.image import ImageDataGenerator
data_aug = ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

In [ ]:
#set the videogenerator for the training process
train_gen = VideoFrameGenerator( 
    glob_pattern=train_dir,
    nb_frames=N_frames,
    split_val=train_validate_split, 
    shuffle=True,     #shuffle the sample for training
    batch_size=Batch_size,
    target_shape=size,
    nb_channel=channels,
    transformation=data_aug,  #apply data augmentaion for the generated frames
    use_frame_cache=False)    

In [ ]:
#set the videogenerator for the validation process
validation_gen= train_gen.get_validation_generator()  

In [ ]:
#show some sample frames generated from videogenerator
from keras_video import utils as ku
ku.show_sample(train_gen, random=True)

## Model Setting of Neural Network: CNN (Fine tuning from MobileNetV2) + GRU + Classifier Layers

In [ ]:
# fine tuning MobileNetV2
def build_mobilenet(shape=(224, 224, 3), nbout=2):
    model = keras.applications.MobileNetV2(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = keras.layers.GlobalMaxPool2D()
    return keras.Sequential([model, output])

In [ ]:
# the sturctue of the Neural Network 
from keras.layers import TimeDistributed, GRU, LSTM, Dense, Dropout
def action_model(shape=(8, 224, 224, 3), nbout=2):
    # create the convnet with (224, 224, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # create the final model
    model = keras.Sequential()
    # add the convnet with (8, 224, 224, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # add GRU
    model.add(GRU(64))
    # add the classification layers
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
from keras import optimizers
Inshape=(N_frames,) + size + (channels,) #(8, 224, 224, 3)
model = action_model(Inshape, classes)
optimizer = optimizers.Adam(0.001)     #Use Adam as optimizers, set learning rate as 0.001
model.compile(
    optimizer,
    'binary_crossentropy',        #use binary crossentropy as the loss function
    metrics=['acc',keras.metrics.FalsePositives(),keras.metrics.FalseNegatives()]
)

## Train the Model and test its performance on validation set

In [ ]:
#training the model
epochs=30                                              # set the epochs=30 since longer training causes over-fitting
callbacks = [            
    keras.callbacks.ModelCheckpoint(
        '/content/gdrive/My Drive/CSCE636_Deep_Learning/Model/chkp/weight_custom.hdf5',   #save the weights with best performance
        verbose=1,
        save_weights_only=True,
        save_best_only=True,),
]
#train the model and see the performance on training and validation set
history=model.fit_generator(
    train_gen,
    validation_data=validation_gen,       
    verbose=1,
    epochs=epochs,
    callbacks=callbacks
)
model.save('/content/gdrive/My Drive/CSCE636_Deep_Learning/Model/chkp/model_custom.h5')       #save the trained model

In [ ]:
#plot the results
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss =history.history['val_loss']

epochs=range(1,len(acc)+1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()